## Insert Data

#### Imports

In [1]:
import pandas as pd
import sqlite3


#### Setup Connection

In [2]:
conn = sqlite3.connect('../data/occupationData.db')
source_dir = '../data/validated_files/'

#### Send data function

In [3]:
columns_to_keep = []
created_at = pd.Timestamp.now()
def insert_data(table, if_exists='replace'):
    try:
       
        if table == 'dimTechSkills':
            columns_to_keep = ['O*NET-SOC Code', 'Example', 'Commodity Code', 'Commodity Title', 'Hot Technology', 'In Demand']
            df = pd.read_csv(f"{source_dir}jobTechSkills.csv", usecols=columns_to_keep)
            df['created_at'] = created_at
            df = df.rename(columns={'O*NET-SOC Code': 'ONET_SOC_Code', 'Commodity Code': 'Commodity_Code', 'Commodity Title': 'Commodity_Title', 'Hot Technology': 'Hot_Tech', 'In Demand': 'In_Demand'})
            df.to_sql(table, conn, if_exists=if_exists, index=False)
            print(f"Inserted {len(df)} records into {table}") 
            
        elif table == 'dimJobInfo':
            columns_to_keep = ['O*NET-SOC Code', 'Title', 'Description']
            df = pd.read_csv(f"{source_dir}jobTitleInfo.csv", usecols=columns_to_keep)
            df['created_at'] = created_at
            df = df.rename(columns={'O*NET-SOC Code': 'ONET_SOC_Code'})
            df.to_sql(table, conn, if_exists=if_exists, index=False)
            print(f"Inserted {len(df)} records into {table}")
            
        elif table == 'dimSkills':
            columns_to_keep = ['O*NET-SOC Code', 'Element ID', 'Element Name', 'Scale ID', 'Data Value']
            df = pd.read_csv(f"{source_dir}jobSkills.csv", usecols=columns_to_keep)
            df['created_at'] = created_at
            df = df.rename(columns={'O*NET-SOC Code': 'ONET_SOC_Code', 'Element ID': 'Element_ID', 'Element Name': 'Skill', 'Scale ID': 'Scale_ID', 'Data_Value': 'Rating'})
            df.to_sql(table, conn, if_exists=if_exists, index=False)
            print(f"Inserted {len(df)} records into {table}")
        
    except FileNotFoundError:
        print(f"File for {source_dir} not found.")
    except Exception as e:
        print(f"Error inserting data into {table}: {e}")

#### Call insert_data function

In [4]:
#insert_data('dimAltTitles')
insert_data('dimTechSkills')
insert_data('dimJobInfo')
insert_data('dimSkills')
#conn.close()

Inserted 32681 records into dimTechSkills
Inserted 1016 records into dimJobInfo


Inserted 34673 records into dimSkills


#### Populate fact table

In [5]:
cursor = conn.cursor()
try:
    cursor.executescript(open('../SQL/factTable_populate.sql').read())
    print("Fact table populated successfully.")
except Exception as e:
    print("Error populating fact table:", e)
    
finally: 
    conn.commit()
    conn.close()

Fact table populated successfully.
